## 📖 Teoria: Classi IP

### Sistema Classful (storico)

```
CLASSE A: 0.0.0.0 - 127.255.255.255
  ┌─ bit 0 = 0
  0xxxxxxx.xxxxxxxx.xxxxxxxx.xxxxxxxx
  Primo ottetto: 0-127
  Netmask: /8
  Uso: Grandi reti (milioni di host)

CLASSE B: 128.0.0.0 - 191.255.255.255
  ┌─ bit 0-1 = 10
  10xxxxxx.xxxxxxxx.xxxxxxxx.xxxxxxxx
  Primo ottetto: 128-191
  Netmask: /16
  Uso: Medie reti (migliaia di host)

CLASSE C: 192.0.0.0 - 223.255.255.255
  ┌─ bit 0-2 = 110
  110xxxxx.xxxxxxxx.xxxxxxxx.xxxxxxxx
  Primo ottetto: 192-223
  Netmask: /24
  Uso: Piccole reti (centinaia di host)

CLASSE D: 224.0.0.0 - 239.255.255.255
  ┌─ bit 0-3 = 1110
  1110xxxx.xxxxxxxx.xxxxxxxx.xxxxxxxx
  Uso: Multicast

CLASSE E: 240.0.0.0 - 255.255.255.255
  ┌─ bit 0-4 = 11110
  1111xxxx.xxxxxxxx.xxxxxxxx.xxxxxxxx
  Uso: Riservato (sperimentale)
```

## 📖 Teoria: Range Speciali

### RFC 1918 - Reti Private

```
10.0.0.0/8        (10.0.0.0 - 10.255.255.255)
  - Classe A privata
  - 16 milioni di host

172.16.0.0/12     (172.16.0.0 - 172.31.255.255)
  - 16 reti Classe B private
  - ~1 milione di host

192.168.0.0/16    (192.168.0.0 - 192.168.255.255)
  - 256 reti Classe C private
  - 65,536 host
```

### RFC 3330 - Altri Range Riservati

```
0.0.0.0/8         - "This network" (sorgente)
127.0.0.0/8       - Loopback (localhost)
169.254.0.0/16    - Link-local (APIPA)
224.0.0.0/4       - Multicast (Classe D)
240.0.0.0/4       - Riservato (Classe E)
255.255.255.255   - Broadcast
```

## 🔢 Algoritmo: Classificazione

### Step 1: Determina Classe

```
Estrai primo ottetto: ip >> 24

if (first_octet < 128)  → CLASSE_A
if (first_octet < 192)  → CLASSE_B
if (first_octet < 224)  → CLASSE_C
if (first_octet < 240)  → CLASSE_D
else                    → CLASSE_E
```

### Step 2: Cerca in Lookup Table

```
SpecialRange[] = {
    {0x00000000, 0xFF000000, RESERVED},  // 0.0.0.0/8
    {0x0A000000, 0xFF000000, PRIVATE},   // 10.0.0.0/8
    {0x7F000000, 0xFF000000, RESERVED},  // 127.0.0.0/8
    ...
};

for each range:
    if (ip & range.netmask) == range.network:
        return range.type

return PUBLIC  // Default
```

## 📝 Implementazione

In [ ]:
%%bash
mkdir -p /home/giordi/Repos/CUgolot/C-Struct/IPscope/tutorials/build/04
cd /home/giordi/Repos/CUgolot/C-Struct/IPscope/tutorials/build/04

# Copia file precedenti
cp ../03/ip_types.h .
cp ../03/ip_utils.* .

# Aggiungi enum e struct per classificazione
cat >> ip_types.h << 'EOF'

// Classi IP (classful addressing)
typedef enum {
    CLASSE_A,      // 0.0.0.0 - 127.255.255.255
    CLASSE_B,      // 128.0.0.0 - 191.255.255.255
    CLASSE_C,      // 192.0.0.0 - 223.255.255.255
    CLASSE_D,      // 224.0.0.0 - 239.255.255.255 (Multicast)
    CLASSE_E       // 240.0.0.0 - 255.255.255.255 (Riservato)
} IPClass;

// Tipi IP per classificazione
typedef enum {
    IP_PUBLIC,     // Indirizzo pubblico routabile
    IP_PRIVATE,    // RFC 1918 (10.x, 172.16-31.x, 192.168.x)
    IP_RESERVED    // Riservato/Speciale (loopback, multicast, etc.)
} IPType;

// Range speciale per lookup
typedef struct {
    uint32_t network;   // Network address
    uint32_t netmask;   // Netmask
    IPType type;        // Tipo del range
    const char *desc;   // Descrizione
} SpecialRange;
EOF

# Aggiungi prototipi funzioni
cat >> ip_utils.h << 'EOF'

// Classificazione
IPClass classify_ip(uint32_t ip);
IPType get_ip_type(uint32_t ip);
const char* ip_type_to_string(IPType type);
const char* ip_class_to_string(IPClass class);
EOF

echo "✅ Header aggiornati"

In [ ]:
%%bash
cd /home/giordi/Repos/CUgolot/C-Struct/IPscope/tutorials/build/04

cat >> ip_utils.c << 'EOF'

/**
 * Lookup table con range speciali RFC
 * Ordinata per specificità (netmask più stretta prima)
 */
static const SpecialRange SPECIAL_RANGES[] = {
    // Broadcast
    {0xFFFFFFFF, 0xFFFFFFFF, IP_RESERVED, "255.255.255.255 - Broadcast"},
    
    // Loopback - 127.0.0.0/8
    {0x7F000000, 0xFF000000, IP_RESERVED, "127.0.0.0/8 - Loopback"},
    
    // Link-local - 169.254.0.0/16 (APIPA)
    {0xA9FE0000, 0xFFFF0000, IP_RESERVED, "169.254.0.0/16 - Link-local"},
    
    // Private - 10.0.0.0/8 (RFC 1918)
    {0x0A000000, 0xFF000000, IP_PRIVATE, "10.0.0.0/8 - Private (RFC 1918)"},
    
    // Private - 172.16.0.0/12 (RFC 1918)
    {0xAC100000, 0xFFF00000, IP_PRIVATE, "172.16.0.0/12 - Private (RFC 1918)"},
    
    // Private - 192.168.0.0/16 (RFC 1918)
    {0xC0A80000, 0xFFFF0000, IP_PRIVATE, "192.168.0.0/16 - Private (RFC 1918)"},
    
    // This network - 0.0.0.0/8
    {0x00000000, 0xFF000000, IP_RESERVED, "0.0.0.0/8 - This network"},
    
    // Multicast - 224.0.0.0/4 (Classe D)
    {0xE0000000, 0xF0000000, IP_RESERVED, "224.0.0.0/4 - Multicast"},
    
    // Riservato - 240.0.0.0/4 (Classe E)
    {0xF0000000, 0xF0000000, IP_RESERVED, "240.0.0.0/4 - Reserved"},
};

static const size_t SPECIAL_RANGES_COUNT = 
    sizeof(SPECIAL_RANGES) / sizeof(SPECIAL_RANGES[0]);

/**
 * Classifica IP secondo sistema classful
 * 
 * Algoritmo: controlla bit più significativi
 * 
 * A: 0xxxxxxx (0-127)
 * B: 10xxxxxx (128-191)
 * C: 110xxxxx (192-223)
 * D: 1110xxxx (224-239)
 * E: 1111xxxx (240-255)
 */
IPClass classify_ip(uint32_t ip) {
    // Estrai primo ottetto
    uint8_t first_octet = ip >> 24;
    
    if (first_octet < 128) return CLASSE_A;
    if (first_octet < 192) return CLASSE_B;
    if (first_octet < 224) return CLASSE_C;
    if (first_octet < 240) return CLASSE_D;
    return CLASSE_E;
}

/**
 * Determina tipo IP (Pubblico/Privato/Riservato)
 * 
 * Algoritmo: cerca in lookup table
 * 
 * Per ogni range speciale:
 *   - Applica netmask a IP
 *   - Confronta con network del range
 *   - Se match, ritorna tipo
 * 
 * Default: IP_PUBLIC
 */
IPType get_ip_type(uint32_t ip) {
    for (size_t i = 0; i < SPECIAL_RANGES_COUNT; i++) {
        const SpecialRange *range = &SPECIAL_RANGES[i];
        
        // Verifica se IP appartiene a questo range
        if ((ip & range->netmask) == range->network) {
            return range->type;
        }
    }
    
    // Default: pubblico
    return IP_PUBLIC;
}

/**
 * Converte IPType a stringa
 */
const char* ip_type_to_string(IPType type) {
    switch (type) {
        case IP_PUBLIC:   return "Pubblico";
        case IP_PRIVATE:  return "Privato";
        case IP_RESERVED: return "Riservato";
        default:          return "Sconosciuto";
    }
}

/**
 * Converte IPClass a stringa
 */
const char* ip_class_to_string(IPClass class) {
    switch (class) {
        case CLASSE_A: return "Classe A";
        case CLASSE_B: return "Classe B";
        case CLASSE_C: return "Classe C";
        case CLASSE_D: return "Classe D (Multicast)";
        case CLASSE_E: return "Classe E (Riservato)";
        default:       return "Sconosciuta";
    }
}
EOF

echo "✅ Funzioni classificazione implementate"

## ✅ Test Classificazione

In [ ]:
%%bash
cd /home/giordi/Repos/CUgolot/C-Struct/IPscope/tutorials/build/04

cat > test_classifier.c << 'EOF'
#include <stdio.h>
#include "ip_utils.h"

void test_classification() {
    printf("=== TEST classify_ip() ===\n\n");
    
    const char *test_ips[] = {
        "8.8.8.8",        // A
        "126.0.0.1",      // A (limite)
        "128.0.0.1",      // B (inizio)
        "172.217.16.46", // B (Google)
        "192.0.2.1",      // C (inizio)
        "223.255.255.255", // C (limite)
        "224.0.0.1",      // D (multicast)
        "239.255.255.255", // D (limite)
        "240.0.0.0",      // E
        "255.255.255.255" // E (broadcast)
    };
    
    for (int i = 0; i < 10; i++) {
        uint32_t ip;
        string_to_ip(test_ips[i], &ip);
        
        IPClass class = classify_ip(ip);
        uint8_t first = ip >> 24;
        
        printf("%-17s (%-3d) → %s\n", 
               test_ips[i], first, ip_class_to_string(class));
    }
    printf("\n");
}

void test_ip_types() {
    printf("=== TEST get_ip_type() ===\n\n");
    
    struct {
        const char *ip;
        const char *desc;
    } tests[] = {
        // Pubblici
        {"8.8.8.8", "Google DNS"},
        {"1.1.1.1", "Cloudflare DNS"},
        {"142.250.180.46", "Google.com"},
        
        // Privati RFC 1918
        {"10.0.0.1", "Privato 10.x"},
        {"172.16.0.1", "Privato 172.16-31.x"},
        {"192.168.1.1", "Privato 192.168.x"},
        
        // Riservati
        {"127.0.0.1", "Loopback"},
        {"0.0.0.0", "This network"},
        {"169.254.1.1", "Link-local"},
        {"224.0.0.1", "Multicast"},
        {"255.255.255.255", "Broadcast"},
    };
    
    for (int i = 0; i < 11; i++) {
        uint32_t ip;
        string_to_ip(tests[i].ip, &ip);
        
        IPType type = get_ip_type(ip);
        
        printf("%-17s → %-10s (%s)\n", 
               tests[i].ip, 
               ip_type_to_string(type),
               tests[i].desc);
    }
    printf("\n");
}

void test_special_ranges() {
    printf("=== ANALISI RANGE PRIVATI ===\n\n");
    
    // Test limiti RFC 1918
    struct {
        const char *range;
        const char *start;
        const char *end;
    } private_ranges[] = {
        {"10.0.0.0/8", "10.0.0.0", "10.255.255.255"},
        {"172.16.0.0/12", "172.16.0.0", "172.31.255.255"},
        {"192.168.0.0/16", "192.168.0.0", "192.168.255.255"},
    };
    
    for (int i = 0; i < 3; i++) {
        printf("%s\n", private_ranges[i].range);
        
        // Test start
        uint32_t start_ip;
        string_to_ip(private_ranges[i].start, &start_ip);
        IPType start_type = get_ip_type(start_ip);
        
        printf("  Start: %s → %s\n", 
               private_ranges[i].start,
               start_type == IP_PRIVATE ? "✓ Privato" : "✗ NON Privato!");
        
        // Test end
        uint32_t end_ip;
        string_to_ip(private_ranges[i].end, &end_ip);
        IPType end_type = get_ip_type(end_ip);
        
        printf("  End:   %s → %s\n\n", 
               private_ranges[i].end,
               end_type == IP_PRIVATE ? "✓ Privato" : "✗ NON Privato!");
    }
}

void test_edge_cases() {
    printf("=== TEST CASI LIMITE ===\n\n");
    
    struct {
        const char *ip;
        const char *note;
    } edges[] = {
        {"172.15.255.255", "Prima di 172.16/12 (pubblico)"},
        {"172.16.0.0", "Inizio 172.16/12 (privato)"},
        {"172.31.255.255", "Fine 172.16/12 (privato)"},
        {"172.32.0.0", "Dopo 172.16/12 (pubblico)"},
    };
    
    for (int i = 0; i < 4; i++) {
        uint32_t ip;
        string_to_ip(edges[i].ip, &ip);
        IPType type = get_ip_type(ip);
        
        printf("%-17s → %-10s\n  %s\n\n", 
               edges[i].ip,
               ip_type_to_string(type),
               edges[i].note);
    }
}

void test_combined_classification() {
    printf("=== CLASSIFICAZIONE COMPLETA ===\n\n");
    
    const char *test_ips[] = {
        "8.8.8.8",
        "10.1.2.3",
        "127.0.0.1",
        "192.168.1.1",
        "224.0.0.1"
    };
    
    printf("%-17s %-20s %-12s\n", "IP", "Classe", "Tipo");
    printf("%-17s %-20s %-12s\n", "──", "──────", "────");
    
    for (int i = 0; i < 5; i++) {
        uint32_t ip;
        string_to_ip(test_ips[i], &ip);
        
        IPClass class = classify_ip(ip);
        IPType type = get_ip_type(ip);
        
        printf("%-17s %-20s %-12s\n", 
               test_ips[i],
               ip_class_to_string(class),
               ip_type_to_string(type));
    }
}

int main() {
    test_classification();
    test_ip_types();
    test_special_ranges();
    test_edge_cases();
    test_combined_classification();
    
    printf("\n✅ Tutti i test completati!\n");
    return 0;
}
EOF

gcc -std=c11 -Wall -Wextra -o test_classifier ip_utils.c test_classifier.c && ./test_classifier

## 🔍 Analisi: Lookup Table Efficiente

### Perché array statico?

```c
static const SpecialRange SPECIAL_RANGES[] = {
    {0xFFFFFFFF, 0xFFFFFFFF, IP_RESERVED, ...},
    // ...
};
```

**Vantaggi**:
- ✅ Nessuna allocazione dinamica
- ✅ Compile-time initialization
- ✅ Cache-friendly (dati contigui)
- ✅ Thread-safe (read-only)

### Algoritmo Lookup

```
for each range in SPECIAL_RANGES:
    masked_ip = ip & range.netmask
    if masked_ip == range.network:
        return range.type
```

**Esempio: 192.168.1.100**

```
Range: 192.168.0.0/16
  network = 0xC0A80000
  netmask = 0xFFFF0000

Test:
  ip      = 0xC0A80164 (192.168.1.100)
  masked  = 0xC0A80164 & 0xFFFF0000
          = 0xC0A80000
  
  0xC0A80000 == 0xC0A80000? ✓ MATCH!
  → IP_PRIVATE
```

## 📊 Tabella RFC 1918

| Range CIDR | Start | End | Host | Uso |
|------------|-------|-----|------|-----|
| **10.0.0.0/8** | 10.0.0.0 | 10.255.255.255 | 16,777,216 | Grandi aziende |
| **172.16.0.0/12** | 172.16.0.0 | 172.31.255.255 | 1,048,576 | Medie aziende |
| **192.168.0.0/16** | 192.168.0.0 | 192.168.255.255 | 65,536 | Home/SOHO |

### Nota: 172.16.0.0/12

```
/12 = primi 12 bit fissi

172.16 = 10101100.00010000
          └───────┘└──┬───┘
             fisso  primi 4 bit fissi (0001)
                    ultimi 4 bit variano (0000-1111)

Secondo ottetto: 16-31
  00010000 = 16
  00011111 = 31

Range: 172.16.0.0 - 172.31.255.255 ✓
```

## 📚 Recap

### ✅ Funzioni implementate:

1. `classify_ip()` - Determina classe A-E
2. `get_ip_type()` - Pubblico/Privato/Riservato
3. `ip_type_to_string()` - Enum → stringa
4. `ip_class_to_string()` - Enum → stringa

### 🎓 Strutture dati:

```c
IPClass: CLASSE_A..E
IPType: IP_PUBLIC/PRIVATE/RESERVED
SpecialRange: {network, netmask, type, desc}
```

### 🔑 Concetti chiave:

- **Classful**: primo ottetto determina classe
- **RFC 1918**: 3 range privati (10.x, 172.16-31.x, 192.168.x)
- **Lookup**: `(ip & netmask) == network`
- **Default**: se non match → PUBLIC

### 🎯 Uso in RANDIP:

```c
// Genera IP casuale
uint32_t ip = random_ip();

// Verifica sia pubblico
if (get_ip_type(ip) == IP_PUBLIC) {
    // OK, usabile!
} else {
    // Scarta, riprova
}
```

### ➡️ Prossimo Notebook

**Notebook 5: IPSet - Gestione Memoria**
- `IPSet` struct (capacità, contatore)
- `ipset_create()` - Allocazione dinamica
- `ipset_add()` - Inserimento con dedup
- `ipset_contains()` - Ricerca lineare
- `ipset_free()` - Cleanup memoria

---

## 🎉 Notebook 4 Completato!

**Funzioni**: 4 funzioni + 9 range speciali  
**Concetti**: Classi IP, RFC 1918/3330, lookup table

Continua con il Notebook 5! 🚀